## Проверка сравнения 2х переводов

In [1]:
import pandas as pd
from scipy.stats import binomtest

In [5]:
data = pd.read_csv("/Users/n.valieva/Downloads/trans_comparison.csv")

In [6]:
data.head(3)

,id,text,translation1,translation2,result
0,406_18,"Exactly, she has to understand that life goes ...","Точно, она должна понять, что жизнь не огранич...","Именно, она должна понять, что жизнь не ограни...",1
1,73_3,I am still having bad flashbacks from War in I...,У меня до сих пор плохие воспоминания о войне ...,У меня все еще плохие флэшбеки от войны в Ирак...,1
2,587_10,I would look to talk to a therapist about thes...,Я бы посоветовал поговорить с терапевтом об эт...,Я бы посоветовал обратиться к терапевту с этим...,2


In [25]:
data.shape

(200, 5)

In [16]:
data_filtered = data[data.result != 3]

In [24]:
n = data_filtered.shape[0]
n

186

In [21]:
new_win = data[data.result == 2].shape[0]
new_win

114

In [22]:
result = binomtest(new_win, n=n, p=0.5, alternative='greater')

In [23]:
result.pvalue

np.float64(0.0012752596263399644)

## Проверка оценки переводов экспертами

In [3]:
import pandas as pd
from scipy.stats import mannwhitneyu

In [5]:
estim = pd.read_csv('/Users/n.valieva/Downloads/translation estimation - оценка перевода.csv')

In [6]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем)
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5


In [11]:
estim['overall'] =  (estim['оценка перевода (Иван)'] + estim['оценка перевода (Екатерина)'] + estim['оценка перевода (Артем)']) / 3
estim['overall'] = estim['overall'].astype(int)

In [17]:
estim.shape

(50, 7)

In [12]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2,3
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1,2
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5,4
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5,4


In [13]:
perplexity_selected_ids = pd.read_csv("top25_bad_perplexity.csv").id
random_selected_ids = pd.read_csv("25_samples_from_all_set.csv").id

In [14]:
perplexity_selected = estim[estim.id.isin(perplexity_selected_ids)]
random_selected = estim[estim.id.isin(random_selected_ids)]

In [16]:
perplexity_selected.shape, random_selected.shape

((25, 7), (25, 7))

In [18]:
mannwhitneyu(perplexity_selected.overall, random_selected.overall)

MannwhitneyuResult(statistic=np.float64(231.0), pvalue=np.float64(0.0938000706682478))

## Проверка оценки переводов моделями

In [9]:
%load_ext autoreload
%autoreload 2 

In [1]:
import sys
import os.path as osp
import os

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
sys.path.append(current_dir + '/notebooks')
sys.path.append(current_dir)
sys.path.append(current_dir + '/data')
sys.path.append(current_dir + '/src')
sys.path.append(current_dir + '/src/utils')
sys.path.append(current_dir + '/src/utils')

In [19]:
import json
from pathlib import Path
from omegaconf import OmegaConf
from datasets import Dataset
from src.core.translate import Translator
from src.utils.schemas import TranslationEstimationResultSchema
import pandas as pd 
from scipy.stats import mannwhitneyu

def read_json(path):
    return json.load(Path(path).open())

def save_json(obj, path):
    json.dump(obj, Path(path).open("w", encoding="utf-8"), indent=4, ensure_ascii=False)
    
def read_file(path: str):
    	return Path(path).open().read()

In [3]:
estim = pd.read_csv('/Users/n.valieva/Downloads/translation estimation - оценка перевода.csv')

In [4]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем)
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5


In [5]:
dataset = Dataset.from_pandas(estim[['id', 'text', 'translation']]).rename_column("translation", "text_rus")

In [15]:
MODELS = ['gpt-4o', 'qwen-2.5-72b-instruct']

In [17]:
for model in MODELS:
	config = OmegaConf.load('../configs/conf.yaml')
	general_translation_config = config.general_translation

	general_translation_config.batch_result_dir = f"batches_res_general_translation"
	general_translation_config.batches = f"batches_general_translation"
	general_translation_config.batch_size = 50

	model_config = read_json(general_translation_config.model_config_path)
	model_config['model_name'] = model
	print(f"Using model: {model}")

	general_translator = Translator(
		system_message=read_file(general_translation_config.prompt_path), 
		model_config=model_config,
		example_data=read_json(general_translation_config.filepath_examples), 
		batch_size=general_translation_config.batch_size,
		batch_result_dir=general_translation_config.batch_result_dir,
		batch_dir=general_translation_config.batches,
		model_type="openai"
	)

	translation_result = general_translator.translate(dataset, TranslationEstimationResultSchema)

	save_json(translation_result, f"translation_result_estimation_{model}.json")

	estim = pd.DataFrame(translation_result).rename(columns={'value': f"value_{model}"}).merge(estim, on='id')

Using model: gpt-4o


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 888.06ba/s]

Processing batch 1/1...



Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 303.41ba/s]


Using model: qwen-2.5-72b-instruct


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 549.21ba/s]


Processing batch 1/1...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1814.93ba/s]


In [27]:
# estim

In [22]:
estim['overall_models'] = sum([estim[f'value_{model}'] for model in MODELS]) / len(MODELS)

In [23]:
estim.head()

,id,value_qwen-2.5-72b-instruct,value_gpt-4o,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall,overall_models
0,812_19,5,5,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2,5.0,5.0
1,900_20,5,5,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3,5.0,5.0
2,431_26,5,5,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1,5.0,5.0
3,1121_19,5,4,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5,4.5,4.5
4,260_25,5,5,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5,5.0,5.0


In [24]:
perplexity_selected_ids = pd.read_csv("top25_bad_perplexity.csv").id
random_selected_ids = pd.read_csv("25_samples_from_all_set.csv").id

In [25]:
perplexity_selected = estim[estim.id.isin(perplexity_selected_ids)]
random_selected = estim[estim.id.isin(random_selected_ids)]

In [26]:
mannwhitneyu(perplexity_selected.overall_models, random_selected.overall_models, )

MannwhitneyuResult(statistic=np.float64(336.0), pvalue=np.float64(0.34266763603504813))